In [1]:
# 2025/12/27
# zhangzhong
# https://docs.langchain.com/oss/python/langgraph/interrupts

In [2]:
# Interrupts allow you to pause graph execution at specific points and wait for external input before continuing
# human in the loop
# When an interrupt is triggered, LangGraph saves the graph state using its persistence layer and waits indefinitely until you resume execution.


In [3]:
## Interrupt 有点像coroutine await 
# - Interrupts work by calling the interrupt() function at any point in your graph nodes.
# - you resume execution by re-invoking the graph using Command, which then becomes the return value of the interrupt() call from inside the node.

In [4]:
# 这个东西可能和我想的收集用户输入的方式不一样，
# - static breakpoint: 我想的功能应该只需要写一个单独的node就可以了 , condition on my graph branches
# - dynamic breakpoint: 但是interupt可以在node的内部中断，是不是一般情况下用不到这个？

# Unlike static breakpoints (which pause before or after specific nodes), interrupts are dynamic
# —they can be placed anywhere in your code and can be conditional based on your application logic.

In [ ]:
## Pause using interupt

from langgraph.types import interrupt

def approval_node(state: State):
    # Pause and ask for approval


    # When you call interrupt, here’s what happens:
    # 1. [Graph execution gets suspended] at the exact point where interrupt is called
    # 2. State is saved using the checkpointer so execution can be resumed later, In production, this should be a persistent checkpointer (e.g. backed by a database)
    # ??? 3. Value is returned to the caller under __interrupt__; it can be any JSON-serializable value (string, object, array, etc.)
    # # Graph waits indefinitely until you resume execution with a response
    # # Response is passed back into the node when you resume, becoming the return value of the interrupt() call
    approved = interrupt("Do you approve this action?")

    # what do you mean?
    # When you resume, Command(resume=...) returns that value here
    return {"approved": approved}

NameError: name 'State' is not defined

In [ ]:
## Resuming interrupts
# After an interrupt pauses execution, you resume the graph by invoking it again with a Command that contains the resume value.

# No! this is not a good idea!
# Key points about resuming:
# You must use the same thread ID when resuming that was used when the interrupt occurred
# The value passed to Command(resume=...) becomes the return value of the interrupt call
# The node restarts from the beginning of the node where the interrupt was called when resumed, so any code before the interrupt runs again
# You can pass any JSON-serializable value as the resume value

## 我觉得human in the loop 的理念是好的，但是总觉得这个interupt设计的不好

from langgraph.types import Command

# Initial run - hits the interrupt and pauses
# thread_id is the persistent pointer (stores a stable ID in production)
config = {"configurable": {"thread_id": "thread-1"}}
result = graph.invoke({"input": "data"}, config=config) # this is the caller

# 那么这么interrupt怎么和现在的聊天api结合呢？ 感觉还是非常的不方便啊！！！
# Check what was interrupted
# __interrupt__ contains the payload that was passed to interrupt()
print(result["__interrupt__"])
# > [Interrupt(value='Do you approve this action?')]

# Resume with the human's response
# The resume payload becomes the return value of interrupt() inside the node
graph.invoke(Command(resume=True), config=config)


# the call approved = interrupt("Do you approve this action?") suspends the graph and 
# returns the __interrupt__ payload to the caller.
#  When you resume that same thread with Command(resume=True), 
# !!! execution restarts from the beginning of the node
# and the call returns the resume value. 

In [ ]:
## Approve or reject
# For example, you might want to ask a human to approve an API call, a database change, or any other important decision.

def approval_node(state: State) -> Command[Literal["proceed", "cancel"]]:
    # 可以看到这些interrupt都是在函数的最开头，避免那个execution restarts from the beginning of the node的问题
    # Pause execution; payload shows up under result["__interrupt__"]
    is_approved = interrupt({
        "question": "Do you want to proceed with this action?",
        "details": state["action_details"]
    })

    # Route based on the response
    if is_approved:
        return Command(goto="proceed")  # Runs after the resume payload is provided
    else:
        return Command(goto="cancel")


# 从interrupt的问题中提取出答案，不就是structured output吗？

In [ ]:
from typing import Literal, Optional, TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt


class ApprovalState(TypedDict):
    action_details: str
    status: Optional[Literal["pending", "approved", "rejected"]]


def approval_node(state: ApprovalState) -> Command[Literal["proceed", "cancel"]]:
    # Expose details so the caller can render them in a UI
    # 所以interrupt的参数值就是graph.invoke在中断的时候的返回值
    decision = interrupt({
        "question": "Approve this action?",
        "details": state["action_details"],
    })

    # Route to the appropriate node after resume
    return Command(goto="proceed" if decision else "cancel")


def proceed_node(state: ApprovalState):
    return {"status": "approved"}


def cancel_node(state: ApprovalState):
    return {"status": "rejected"}


builder = StateGraph(ApprovalState)
builder.add_node("approval", approval_node)
builder.add_node("proceed", proceed_node)
builder.add_node("cancel", cancel_node)
builder.add_edge(START, "approval")
builder.add_edge("proceed", END)
builder.add_edge("cancel", END)

# Use a more durable checkpointer in production
checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "approval-123"}}
initial = graph.invoke(
    {"action_details": "Transfer $500", "status": "pending"},
    config=config,
)
# 我们是需要拿着这个问题，去问人类，然后从人类的回答中，结构化提取出我们需要的类型，再传给interrupt函数的
print(initial["__interrupt__"])  # -> [Interrupt(value={'question': ..., 'details': ...})]

# Resume with the decision; True routes to proceed, False to cancel
resumed = graph.invoke(Command(resume=True), config=config)
print(resumed["status"])  # -> "approved"

[Interrupt(value={'question': 'Approve this action?', 'details': 'Transfer $500'}, id='216b11498506780b705177567fbe089c')]
approved


In [ ]:
## Comman patterns

# Approval workflows: Pause before executing critical actions (API calls, database changes, financial transactions)
# Review and edit: Let humans review and modify LLM outputs or tool calls before continuing
# Interrupting tool calls: Pause before executing tool calls to review and edit the tool call before execution
# Validating human input: Pause before proceeding to the next step to validate human input

In [ ]:
## Approve or reject

from typing import Literal
from langgraph.types import interrupt, Command

def approval_node(state: State) -> Command[Literal["proceed", "cancel"]]:
    # Pause execution; payload shows up under result["__interrupt__"]
    is_approved = interrupt({
        "question": "Do you want to proceed with this action?",
        "details": state["action_details"]
    })

    # Route based on the response
    if is_approved:
        return Command(goto="proceed")  # Runs after the resume payload is provided
    else:
        return Command(goto="cancel")
    
# When you resume the graph, pass true to approve or false to reject:
# To approve
graph.invoke(Command(resume=True), config=config)

# To reject
graph.invoke(Command(resume=False), config=config)

In [ ]:
## Review and edit state

from langgraph.types import interrupt

def review_node(state: State):
    # Pause and show the current content for review (surfaces in result["__interrupt__"])
    edited_content = interrupt({
        "instruction": "Review and edit this content",
        "content": state["generated_text"]
    })

    # Update the state with the edited version
    return {"generated_text": edited_content}

# ...

graph.invoke(
    Command(resume="The edited and improved text"),  # Value becomes the return from interrupt()
    config=config
)

In [ ]:
## Interrupts in tools

from langchain.tools import tool
from langgraph.types import interrupt

# 这明显是反模式的啊！！！怎么能把interrupt和tool的逻辑放到一起呢？
# 我怎么对tool进行测试呢？如果我突然不想在这个工具里interrupt了呢？
# 如果这个工具需要在其他地方被使用呢？
@tool
def send_email(to: str, subject: str, body: str):
    """Send an email to a recipient."""

    # Pause before sending; payload surfaces in result["__interrupt__"]
    response = interrupt({
        "action": "send_email",
        "to": to,
        "subject": subject,
        "body": body,
        "message": "Approve sending this email?"
    })

    if response.get("action") == "approve":
        # Resume value can override inputs before executing
        final_to = response.get("to", to)
        final_subject = response.get("subject", subject)
        final_body = response.get("body", body)
        return f"Email sent to {final_to} with subject '{final_subject}'"
    return "Email cancelled by user"

In [ ]:
## Validate human input
# Sometimes you need to validate input from humans and ask again if it’s invalid. 


# 真的可以给智能体设计一种编程语言！
# 比如我突然想到的，这个human in the loop，现在是完全做不到递归的，就是他一次只能问一个问题
# 如果在用户回答某个问题的过程中，又有其他问题需要用户确认，就需要像现在的编程语言一样
# 启动一个新的stack layer，但是现在的langgraph显然是做不到的
# 这些功能越思考就越像一门编程语言！

from langgraph.types import interrupt

def get_age_node(state: State):
    prompt = "What is your age?"

    while True:
        answer = interrupt(prompt)  # payload surfaces in result["__interrupt__"]

        # Validate the input
        if isinstance(answer, int) and answer > 0:
            # Valid input - continue
            break
        else:
            # Invalid input - ask again with a more specific prompt
            prompt = f"'{answer}' is not a valid age. Please enter a positive number."

    return {"age": answer}


In [ ]:
## 由于langgraph的interrupt的设计缺陷，我决定不使用interrupt！
# https://docs.langchain.com/oss/python/langgraph/interrupts#rules-of-interrupts
# When you call interrupt within a node, LangGraph suspends execution by raising an exception that signals the runtime to pause. 
# This exception propagates up through the call stack and is caught by the runtime, 
# which notifies the graph to save the current state and wait for external input.

# When execution resumes (after you provide the requested input), 
# the runtime restarts the entire node from the beginning—it does not resume from the exact line where interrupt was called.
# This means any code that ran before the interrupt will execute again. 
# Because of this, there’s a few important rules to follow when working with interrupts to ensure they behave as expected.

## Tips
# 1. Do not wrap interrupt calls in try/except
# 2. Do not reorder interrupt calls within a node
# 3. Do not return complex values in interrupt calls
# 4. Side effects called before interrupt must be idempotent